# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Clouds (Numberic)
0,0,puerto natales,-51.7236,-72.4875,57.65,47,broken clouds,18.41,CL,2024-12-09 12:05:18,75
1,1,port elizabeth,-33.9180,25.5701,75.49,90,scattered clouds,6.91,ZA,2024-12-09 12:05:19,40
2,2,ust-nera,64.5667,143.2000,-6.74,99,overcast clouds,3.31,RU,2024-12-09 12:05:20,100
3,3,boyolangu,-8.1181,111.8935,77.52,91,overcast clouds,2.35,ID,2024-12-09 12:05:21,100
4,4,lindley,-27.8791,27.9135,78.49,21,clear sky,2.35,ZA,2024-12-09 12:05:22,2


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    title = "City Map Plot",
    color = "City",
    legend = "right"
)

map_plot




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[
    (city_data_df[ 'Max Temp'] < 85) &
    (city_data_df['Max Temp']> 60) &
    (city_data_df['Humidity'] < 50) & 
    (city_data_df['Humidity'] > 20) & 
    (city_data_df["Wind Speed"]< 15) & 
    (city_data_df['Clouds (Numberic)']<20)]

# Drop any rows with null values
ideal_weather.dropna(inplace=True)

# Display sample data
ideal_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Clouds (Numberic)
4,4,lindley,-27.8791,27.9135,78.49,21,clear sky,2.35,ZA,2024-12-09 12:05:22,2
172,172,miahuatlan de porfirio diaz,16.3333,-96.6000,74.61,39,clear sky,4.83,MX,2024-12-09 12:08:38,0
192,192,mbaiki,3.8678,17.9892,82.89,33,clear sky,2.44,CF,2024-12-09 12:09:02,4
211,211,bredasdorp,-34.5322,20.0403,72.55,43,few clouds,10.07,ZA,2024-12-09 12:09:22,18
229,229,villa juarez,24.6536,-107.5411,75.45,38,clear sky,6.91,MX,2024-12-09 12:09:42,0
235,235,adam,22.3756,57.5233,71.89,33,clear sky,6.49,OM,2024-12-09 12:09:49,1
245,245,mar del plata,-38.0023,-57.5575,72.52,33,clear sky,10.36,AR,2024-12-09 12:05:23,0
335,335,nahiyat ash shinafiyah,31.5874,44.6602,63.43,46,clear sky,12.73,IQ,2024-12-09 12:11:47,0
385,385,rawson,-43.3002,-65.1023,68.29,42,clear sky,12.66,AR,2024-12-09 12:08:37,0
396,396,valparaiso,-33.0393,-71.6273,79.03,43,clear sky,13.80,CL,2024-12-09 12:09:26,0


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,lindley,ZA,-27.8791,27.9135,21,
172,miahuatlan de porfirio diaz,MX,16.3333,-96.6000,39,
192,mbaiki,CF,3.8678,17.9892,33,
211,bredasdorp,ZA,-34.5322,20.0403,43,
229,villa juarez,MX,24.6536,-107.5411,38,
235,adam,OM,22.3756,57.5233,33,
245,mar del plata,AR,-38.0023,-57.5575,33,
335,nahiyat ash shinafiyah,IQ,31.5874,44.6602,46,
385,rawson,AR,-43.3002,-65.1023,42,
396,valparaiso,CL,-33.0393,-71.6273,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters for hotel search
radius = 10000
categories = "accommodation.hotel"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    city_lat = row['Lat']
    city_lng = row['Lng']

    # Define parameters for the API request
    params = {
        "filter": f"circle:{city_lng},{city_lat},{radius}",
        "bias": f"proximity:{city_lng},{city_lat}",
        "categories": categories,
        "apiKey": geoapify_key
    }

    # Print debug info
    print("Request parameters:", params)

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

   # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Request parameters: {'filter': 'circle:27.9135,-27.8791,10000', 'bias': 'proximity:27.9135,-27.8791', 'categories': 'accommodation.hotel', 'apiKey': '39479faf6fc948c992b87ac951b704bb'}
lindley - nearest hotel: No hotel found
Request parameters: {'filter': 'circle:-96.6,16.3333,10000', 'bias': 'proximity:-96.6,16.3333', 'categories': 'accommodation.hotel', 'apiKey': '39479faf6fc948c992b87ac951b704bb'}
miahuatlan de porfirio diaz - nearest hotel: No hotel found
Request parameters: {'filter': 'circle:17.9892,3.8678,10000', 'bias': 'proximity:17.9892,3.8678', 'categories': 'accommodation.hotel', 'apiKey': '39479faf6fc948c992b87ac951b704bb'}
mbaiki - nearest hotel: No hotel found
Request parameters: {'filter': 'circle:20.0403,-34.5322,10000', 'bias': 'proximity:20.0403,-34.5322', 'categories': 'accommodation.hotel', 'apiKey': '39479faf6fc948c992b87ac951b704bb'}
bredasdorp - nearest hotel: Victoria Hotel
Request parameters: {'filter': 'circle:-107.5411,24.6536,10000', 'bias': 'proximity:-107

,City,Country,Lat,Lng,Humidity,Hotel Name
4,lindley,ZA,-27.8791,27.9135,21,No hotel found
172,miahuatlan de porfirio diaz,MX,16.3333,-96.6000,39,No hotel found
192,mbaiki,CF,3.8678,17.9892,33,No hotel found
211,bredasdorp,ZA,-34.5322,20.0403,43,Victoria Hotel
229,villa juarez,MX,24.6536,-107.5411,38,No hotel found
235,adam,OM,22.3756,57.5233,33,No hotel found
245,mar del plata,AR,-38.0023,-57.5575,33,Hotel 3 de Abril
335,nahiyat ash shinafiyah,IQ,31.5874,44.6602,46,No hotel found
385,rawson,AR,-43.3002,-65.1023,42,Hotel Deportivo
396,valparaiso,CL,-33.0393,-71.6273,43,Ibis Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    title = "City Map Plot",
    color = "City",
    legend = "right",
    hover_cols = ['Hotel Name', 'Country']
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)